<a href="https://colab.research.google.com/github/aisha-partha/AIMLOps-Assignments/blob/main/Experimentation_Phase_2_Pipeline_Building.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Certification Programme in AI and MLOps
## A programme by IISc and TalentSprint
### Assignment: Research to Production Environment: Experimentation_Phase : 2. Pipeline_Building


## Learning Objectives


At the end of the experiment you will be able to:
* create custom classes required for processing  
* implement pipeline and train the model
* save the model



In [1]:
# @title Download the datasets
from IPython import get_ipython

ipython = get_ipython()

notebook="U1_MH1_Data_Munging" #name of the notebook

def setup():
    from IPython.display import HTML, display
    ipython.magic("sx wget https://cdn.iiith.talentsprint.com/aiml/Experiment_related_data/titanic.csv")
    ipython.magic("sx wget https://cdn.iiith.talentsprint.com/aiml/Experiment_related_data/test_titanic.csv")
    print("Data downloaded successfully")
    return

setup()

Data downloaded successfully


In [4]:
!ls

sample_data  test_titanic.csv  titanic.csv


List of files present

### Setup Steps:

In [2]:
#@title Please enter your registration id to start: { run: "auto", display-mode: "form" }
Id = "2304896" #@param {type:"string"}

In [3]:
#@title Please enter your password (your registered phone number) to continue: { run: "auto", display-mode: "form" }
password = "9916583736" #@param {type:"string"}

In [5]:
#@title Run this cell to complete the setup for this Notebook
from IPython import get_ipython

ipython = get_ipython()

notebook= "Experimentation_Phase_2_Pipeline_Building" #name of the notebook

def setup():
#  ipython.magic("sx pip3 install torch")

    from IPython.display import HTML, display
    display(HTML('<script src="https://dashboard.talentsprint.com/aiml/record_ip.html?traineeId={0}&recordId={1}"></script>'.format(getId(),submission_id)))
    print("Setup completed successfully")
    return

def submit_notebook():
    ipython.magic("notebook -e "+ notebook + ".ipynb")

    import requests, json, base64, datetime

    url = "https://dashboard.talentsprint.com/xp/app/save_notebook_attempts"
    if not submission_id:
      data = {"id" : getId(), "notebook" : notebook, "mobile" : getPassword()}
      r = requests.post(url, data = data)
      r = json.loads(r.text)

      if r["status"] == "Success":
          return r["record_id"]
      elif "err" in r:
        print(r["err"])
        return None
      else:
        print ("Something is wrong, the notebook will not be submitted for grading")
        return None

    elif getAnswer() and getComplexity() and getAdditional() and getConcepts() and getComments() and getMentorSupport():
      f = open(notebook + ".ipynb", "rb")
      file_hash = base64.b64encode(f.read())

      data = {"complexity" : Complexity, "additional" :Additional,
              "concepts" : Concepts, "record_id" : submission_id,
              "answer" : Answer, "id" : Id, "file_hash" : file_hash,
              "notebook" : notebook,
              "feedback_experiments_input" : Comments,
              "feedback_mentor_support": Mentor_support}
      r = requests.post(url, data = data)
      r = json.loads(r.text)
      if "err" in r:
        print(r["err"])
        return None
      else:
        print("Your submission is successful.")
        print("Ref Id:", submission_id)
        print("Date of submission: ", r["date"])
        print("Time of submission: ", r["time"])
        print("View your submissions: https://aimlops-iisc.talentsprint.com/notebook_submissions")
        #print("For any queries/discrepancies, please connect with mentors through the chat icon in LMS dashboard.")
        return submission_id
    else: submission_id


def getAdditional():
  try:
    if not Additional:
      raise NameError
    else:
      return Additional
  except NameError:
    print ("Please answer Additional Question")
    return None

def getComplexity():
  try:
    if not Complexity:
      raise NameError
    else:
      return Complexity
  except NameError:
    print ("Please answer Complexity Question")
    return None

def getConcepts():
  try:
    if not Concepts:
      raise NameError
    else:
      return Concepts
  except NameError:
    print ("Please answer Concepts Question")
    return None


# def getWalkthrough():
#   try:
#     if not Walkthrough:
#       raise NameError
#     else:
#       return Walkthrough
#   except NameError:
#     print ("Please answer Walkthrough Question")
#     return None

def getComments():
  try:
    if not Comments:
      raise NameError
    else:
      return Comments
  except NameError:
    print ("Please answer Comments Question")
    return None


def getMentorSupport():
  try:
    if not Mentor_support:
      raise NameError
    else:
      return Mentor_support
  except NameError:
    print ("Please answer Mentor support Question")
    return None

def getAnswer():
  try:
    if not Answer:
      raise NameError
    else:
      return Answer
  except NameError:
    print ("Please answer Question")
    return None


def getId():
  try:
    return Id if Id else None
  except NameError:
    return None

def getPassword():
  try:
    return password if password else None
  except NameError:
    return None

submission_id = None
### Setup
if getPassword() and getId():
  submission_id = submit_notebook()
  if submission_id:
    setup()
else:
  print ("Please complete Id and Password cells before running setup")



Setup completed successfully


## Import Required Packages

In [6]:
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
# to persist the model and the scaler
import joblib
from typing import List
# import for pipeline
from sklearn.pipeline import Pipeline
# for the preprocessors
from sklearn.base import BaseEstimator, TransformerMixin

### **1. Pre-Pipeline-Steps: Load, Explore and Prepare the Data Set**

* Understand different features in the training dataset
* Understand the data types of each columns
* Notice the columns of missing values




In [7]:
# Load the dataset
data = pd.read_csv("titanic.csv")

In [8]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [9]:
# Getting information about the dataset
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


## Data Cleaning and Processing


 ### 1.1 Working on "SibSp" & "Parch" columns:
Combine columns "SibSp" & "Parch" and create another column that represents the total passengers in one ticket with the name "family_size". In each ticket, there might be Siblings/Spouses (SibSp =Number of Siblings/Spouses Aboard) or Parents/Children (Parch=Number of Parents/Children Aboard ) along with the passenger who booked the ticket.

  

In [10]:
 def family_size(data_frame):
   df=data_frame.copy()
   df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
   return df

In [11]:
data=family_size(data)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
 12  FamilySize   891 non-null    int64  
dtypes: float64(2), int64(6), object(5)
memory usage: 90.6+ KB


### 1.2 Working on "Cabin" column:
Find unique entries in the Cabin column. We can label all passengers in two categories having a cabin or not. Check the data type(use: type) of each entry of the Cabin. Convert a string data type into '1' i.e. passengers with cabin and others into '0' i.e. passengers without cabin.  Write a function for the above operation and apply it to the cabin column and create another column with the name " Has_cabin" containing only 0 or 1 entries.





In [12]:
def process_cabin(data_frame):
  df=data_frame.copy()
  f1=lambda x: 0 if type(x) == float else 1 ## Ternary Expression
  df['Has_cabin']=df['Cabin'].apply(f1)
  return df

In [13]:
data=process_cabin(data)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
 12  FamilySize   891 non-null    int64  
 13  Has_cabin    891 non-null    int64  
dtypes: float64(2), int64(7), object(5)
memory usage: 97.6+ KB


### 1.3  Working on "Name" column :
Fetch titles from the name. We can map these titles with numbers and convert them into an integer by mapping with relative numbers. Use: concept of the regular expression.

In [14]:
# extracts the title (Mr, Ms, etc) from the name variable

def get_title(passenger):
    line = passenger
    if re.search('Mrs', line):
        return 'Mrs'
    elif re.search('Mr', line):
        return 'Mr'
    elif re.search('Miss', line):
        return 'Miss'
    elif re.search('Master', line):
        return 'Master'
    else:
        return 'Other'

In [15]:
#Example:
print(get_title('Heranld Mr.'))
data['Name'].apply(get_title)

Mr


0         Mr
1        Mrs
2       Miss
3        Mrs
4         Mr
       ...  
886    Other
887     Miss
888     Miss
889       Mr
890       Mr
Name: Name, Length: 891, dtype: object

In [16]:
data['Title'] = data['Name'].apply(get_title)

In [17]:
print(data['Title'].unique())
data.info()

['Mr' 'Mrs' 'Miss' 'Master' 'Other']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
 12  FamilySize   891 non-null    int64  
 13  Has_cabin    891 non-null    int64  
 14  Title        891 non-null    object 
dtypes: float64(2), int64(7), object(6)
memory usage: 104.5+ KB


In [18]:
# drop unnecessary variables
data.drop(labels=['PassengerId','Name','SibSp','Parch','Ticket','Cabin'], axis=1, inplace=True)

# display data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Survived    891 non-null    int64  
 1   Pclass      891 non-null    int64  
 2   Sex         891 non-null    object 
 3   Age         714 non-null    float64
 4   Fare        891 non-null    float64
 5   Embarked    889 non-null    object 
 6   FamilySize  891 non-null    int64  
 7   Has_cabin   891 non-null    int64  
 8   Title       891 non-null    object 
dtypes: float64(2), int64(4), object(3)
memory usage: 62.8+ KB


### **2. Pipeline-Steps**
### Building custom class compatible with sklearn pipeline for imputation, feature mapping and any specific operation on any column.

### **A. Imputation**

Buiding custom Imputation class compatible with Sklearn for 'Embarked' colum imputation.

In [19]:
class embarkImputer(BaseEstimator, TransformerMixin):
    """embarked column imputer."""

    def __init__(self, variables: str):

        if not isinstance(variables, str):
            raise ValueError("variables should be a list")

        self.variables = variables

    def fit(self, X: pd.DataFrame, y: pd.Series = None):
        # we need the fit statement to accomodate the sklearn pipeline
        self.fill_value=X[self.variables].mode()[0]
        return self

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        X = X.copy()
        X[self.variables]=X[self.variables].fillna(self.fill_value)

        return X

In [20]:
data.Embarked.isnull().sum()

2

In [21]:
emb=embarkImputer('Embarked')
emb.fit(data)
print(len(emb.transform(data).Embarked))
data1=emb.transform(data)
print(len(data1.Embarked))
print(data1.Embarked.isnull().sum())


891
891
0


In [22]:
data1.Embarked.dtypes

dtype('O')

In [23]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Survived    891 non-null    int64  
 1   Pclass      891 non-null    int64  
 2   Sex         891 non-null    object 
 3   Age         714 non-null    float64
 4   Fare        891 non-null    float64
 5   Embarked    891 non-null    object 
 6   FamilySize  891 non-null    int64  
 7   Has_cabin   891 non-null    int64  
 8   Title       891 non-null    object 
dtypes: float64(2), int64(4), object(3)
memory usage: 62.8+ KB


### **B. Mapping**

Bulding Mapper class for mpping 'Embarked','Sex' and 'Title' columns

In [24]:
class Mapper(BaseEstimator, TransformerMixin):
    """Categorical variable mapper."""

    def __init__(self, variables: str, mappings: dict):

        if not isinstance(variables, str):
            raise ValueError("variables should be a str")

        self.variables = variables
        self.mappings = mappings

    def fit(self, X: pd.DataFrame, y: pd.Series = None):
        # we need the fit statement to accomodate the sklearn pipeline
        return self

    def transform(self, X: pd.DataFrame) -> pd.DataFrame:
        X = X.copy()
        #for feature in self.variables:
        X[self.variables] = X[self.variables].map(self.mappings).astype(int)

        return X

In [25]:
map_sex=Mapper('Sex',{'female': 0, 'male': 1})
map_embarked=Mapper('Embarked', {'S': 0, 'C': 1, 'Q': 2} )
map_title=Mapper('Title', {'Mrs': 4, 'Master':3 ,'Miss': 2, 'Mr': 1,'Other':0} )

In [26]:
data1.head()

,Survived,Pclass,Sex,Age,Fare,Embarked,FamilySize,Has_cabin,Title
0,0,3,male,22.0,7.2500,S,2,0,Mr
1,1,1,female,38.0,71.2833,C,2,1,Mrs
2,1,3,female,26.0,7.9250,S,1,0,Miss
3,1,1,female,35.0,53.1000,S,2,1,Mrs
4,0,3,male,35.0,8.0500,S,1,0,Mr


In [27]:
for i in [map_sex,map_embarked,map_title]:
  data1=i.fit(data).transform(data1)

In [28]:
data1

,Survived,Pclass,Sex,Age,Fare,Embarked,FamilySize,Has_cabin,Title
0,0,3,1,22.0,7.2500,0,2,0,1
1,1,1,0,38.0,71.2833,1,2,1,4
2,1,3,0,26.0,7.9250,0,1,0,2
3,1,1,0,35.0,53.1000,0,2,1,4
4,0,3,1,35.0,8.0500,0,1,0,1
...,...,...,...,...,...,...,...,...,...
886,0,2,1,27.0,13.0000,0,1,0,0
887,1,1,0,19.0,30.0000,0,1,1,2
888,0,3,0,NaN,23.4500,0,4,0,2
889,1,1,1,26.0,30.0000,1,1,1,1


In [29]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Survived    891 non-null    int64  
 1   Pclass      891 non-null    int64  
 2   Sex         891 non-null    object 
 3   Age         714 non-null    float64
 4   Fare        891 non-null    float64
 5   Embarked    889 non-null    object 
 6   FamilySize  891 non-null    int64  
 7   Has_cabin   891 non-null    int64  
 8   Title       891 non-null    object 
dtypes: float64(2), int64(4), object(3)
memory usage: 62.8+ KB


### **C. Class for Specific operation : Age column transformation**

Creating Class for Age column transformation that is compatible with SK_learn pipeline:

In the pre-processing steps , a function was created for processing age column. Now we are converting that function into a class suitable for inserting inside the pipeline.

In [30]:
#from sklearn.base import BaseEstimator, TransformerMixin

class age_col_tfr(BaseEstimator, TransformerMixin):

  """Age column transformer"""

  def __init__(self, variables):

    if not isinstance(variables, str):
      raise ValueError('variables should be a str')
    self.variables = variables

  def fit(self, X: pd.DataFrame, y=None):
    self.age_avg = X[self.variables].mean()
    self.age_std = X[self.variables].std()
      # we need this step to fit the sklearn pipeline
    return self

  def transform(self, X):
    np.random.seed(42)
    X = X.copy() # so that we do not over-write the original dataframe
    age_null_count = X[self.variables].isnull().sum()
    age_null_random_list = np.random.randint(self.age_avg - self.age_std, self.age_avg + self.age_std, size=age_null_count)
    X.loc[np.isnan(X[self.variables]),self.variables] = age_null_random_list
    X[self.variables] = X[self.variables].astype(int)
    return X


In [31]:
age_tfr=age_col_tfr('Age')

In [32]:
age_tfr.fit(data1)

age_col_tfr(variables='Age')

In [33]:
data1=age_tfr.transform(data1)

In [34]:
sum(data1.Age.isnull())

0

### **3. Building Pipeline**

Finally building pipeline and implementing all above class inside pipeline along with classifier also inside.

In [35]:
titanic_pipe = Pipeline([

    ('embark_imputation', embarkImputer(variables='Embarked')),
    ##==========Mapper======##
    ('map_sex',Mapper('Sex',{'female': 0, 'male': 1})),
    ('map_embarked',Mapper('Embarked', {'S': 0, 'C': 1, 'Q': 2} )),
    ('map_title',Mapper('Title', {'Mrs': 4, 'Master':3 ,'Miss': 2, 'Mr': 1,'Other':0})),

    # Transformation of age column
    ('age_transform', age_col_tfr(variables='Age')),

    # scale
    ('scaler', StandardScaler()),

    # Model fit
    ('model_rf', RandomForestClassifier(n_estimators=150, max_depth=5,random_state=42))
])

### Train_test_split

In [36]:
x=data.drop('Survived', axis=1)
y=data['Survived']
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=42)
X_train.shape, X_test.shape

((712, 8), (179, 8))

#### Try to understand missing index in age column in test set

In [37]:
X_test.loc[X_test.Age.isnull(),'Age'].index

Index([709, 300, 485, 653, 538, 793, 235, 198, 739, 837, 790, 168, 196, 778,
       250, 388,  76, 669, 849, 527, 578, 359,  65, 481, 306,  31, 584, 109,
        77, 611, 643,  82, 507, 375,   5, 457, 773],
      dtype='int64')

In [38]:
np.isnan(X_test.loc[709,'Age'])

True

Using age_tfr class to fill the missing entires

In [39]:
age_tfr=age_col_tfr('Age')

In [40]:
#age_tfr.fit(X_train)
age_tfr.fit(X_test)

age_col_tfr(variables='Age')

In [41]:
age_column_filled=age_tfr.transform(X_test)
age_column_filled.loc[709,'Age']

21

### Training the model using pipeline above

In [42]:
titanic_pipe.fit(X_train,y_train)
y_pred = titanic_pipe.predict(X_test)
# Calculate the accuracy
print("Accuracy(in %):", accuracy_score(y_test, y_pred)*100)

Accuracy(in %): 82.12290502793296


### Persist the model

In [43]:
joblib.dump(titanic_pipe,'titanic_pip.joblib')

['titanic_pip.joblib']

### Checking for the versions may be used for requirements.txt file

In [44]:
import numpy as np
import pandas as pd
import sklearn
import pydantic
import joblib

In [45]:
pip install pydantic

In [46]:
pip install strictyaml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.9/123.9 kB 1.4 MB/s eta 0:00:00


In [47]:
pip install ruamel.yaml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 5.5 MB/s eta 0:00:00


In [48]:
import strictyaml

In [49]:
import ruamel.yaml


In [50]:
print(np.__version__)
print(pd.__version__)
print(sklearn.__version__)
print(pydantic.__version__)
print(strictyaml.__version__)
print(ruamel.yaml.__version__)
print(joblib.__version__)

1.25.2
2.0.3
1.2.2
2.7.4
1.6.2
0.18.6
1.4.2


### Please answer the questions below to complete the experiment:




In [51]:
#@title  In the final pipeline, in the notebook, what happens while applying the 'train' and 'predict' methods? **Train:** It learns parameters viz. mean, standard deviation, and others as per the transformer classes along with trained weights and biases via ML model, using the dataset passed as an argument. **Predict:** It applies those parameters learned from the dataset used in the 'train' method and transforms the dataset passed inside the 'predict' method. After transformation, it applies the 'predict' method from the ML model. {run: "auto", form-width: "500px", display-mode: "form" }
Answer = "True" #@param [" ", "True", "False"]

In [52]:
#@title How was the experiment? { run: "auto", form-width: "500px", display-mode: "form" }
Complexity = "Good and Challenging for me" #@param ["","Too Simple, I am wasting time", "Good, But Not Challenging for me", "Good and Challenging for me", "Was Tough, but I did it", "Too Difficult for me"]


In [53]:
#@title If it was too easy, what more would you have liked to be added? If it was very difficult, what would you have liked to have been removed? { run: "auto", display-mode: "form" }
Additional = "na" #@param {type:"string"}


In [54]:
#@title Can you identify the concepts from the lecture which this experiment covered? { run: "auto", vertical-output: true, display-mode: "form" }
Concepts = "Yes" #@param ["","Yes", "No"]


In [55]:
#@title  Text and image description/explanation and code comments within the experiment: { run: "auto", vertical-output: true, display-mode: "form" }
Comments = "Very Useful" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [56]:
#@title Mentor Support: { run: "auto", vertical-output: true, display-mode: "form" }
Mentor_support = "Very Useful" #@param ["","Very Useful", "Somewhat Useful", "Not Useful", "Didn't use"]


In [57]:
#@title Run this cell to submit your notebook for grading { vertical-output: true }
try:
  if submission_id:
      return_id = submit_notebook()
      if return_id : submission_id = return_id
  else:
      print("Please complete the setup first.")
except NameError:
  print ("Please complete the setup first.")

Your submission is successful.
Ref Id: 4829
Date of submission:  22 Jun 2024
Time of submission:  20:58:09
View your submissions: https://aimlops-iisc.talentsprint.com/notebook_submissions
